In [1]:
import pandas as pd
data=pd.read_csv("C:\\Users\\Aarav Menon\\OneDrive\\Desktop\\project\\journal datastd\\DET_EXPRESS2009_3_4_5数据-03.csv")
print(data.head())

   LINK_ID                 TIME  FLOW  LANE_NO  OCC  ASPEED  LARGE_CAR_FLOW
0     3036  2009-06-01 00:01:13     6       13    1      77               0
1     3036  2009-06-01 00:01:13    13       12    2      83               2
2     3036  2009-06-01 00:01:13    16       11    2      68               1
3     3036  2009-06-01 00:01:13     9        3    1      79               0
4     3036  2009-06-01 00:01:13    19        2    4      47               3


In [2]:
print(data.isnull().sum())

LINK_ID           0
TIME              0
FLOW              0
LANE_NO           0
OCC               0
ASPEED            0
LARGE_CAR_FLOW    0
dtype: int64


In [3]:
#normalize
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
numerical_features=['FLOW','OCC','ASPEED','LARGE_CAR_FLOW']
data[numerical_features]=scaler.fit_transform(data[numerical_features])
data

,LINK_ID,TIME,FLOW,LANE_NO,OCC,ASPEED,LARGE_CAR_FLOW
0,3036,2009-06-01 00:01:13,-1.184004,13,-0.658748,1.161728,-0.705976
1,3036,2009-06-01 00:01:13,-0.791068,12,-0.567531,1.523713,-0.270084
2,3036,2009-06-01 00:01:13,-0.622667,11,-0.567531,0.618751,-0.488030
3,3036,2009-06-01 00:01:13,-1.015603,3,-0.658748,1.282390,-0.705976
4,3036,2009-06-01 00:01:13,-0.454266,2,-0.385096,-0.648197,-0.052137
...,...,...,...,...,...,...,...
67786,3036,2009-06-22 14:20:26,0.612274,2,-0.111444,0.015442,-0.488030
67787,3036,2009-06-22 14:20:26,0.892942,1,0.070991,0.256766,-0.052137
67788,3036,2009-06-22 14:20:26,0.443873,13,0.527078,-0.225881,1.037594
67789,3036,2009-06-22 14:20:26,1.285878,12,0.618295,0.136104,0.819648


In [4]:
#removing date and other columns
data.drop(columns=['LANE_NO'],inplace=True)
data.drop(columns=['TIME'],inplace=True)
data.drop(columns=['LINK_ID'],inplace=True)
data

,FLOW,OCC,ASPEED,LARGE_CAR_FLOW
0,-1.184004,-0.658748,1.161728,-0.705976
1,-0.791068,-0.567531,1.523713,-0.270084
2,-0.622667,-0.567531,0.618751,-0.488030
3,-1.015603,-0.658748,1.282390,-0.705976
4,-0.454266,-0.385096,-0.648197,-0.052137
...,...,...,...,...
67786,0.612274,-0.111444,0.015442,-0.488030
67787,0.892942,0.070991,0.256766,-0.052137
67788,0.443873,0.527078,-0.225881,1.037594
67789,1.285878,0.618295,0.136104,0.819648


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

# Define the number of features in your dataset
num_features = len(data.columns) - 1  # Exclude the target variable (FLOW)

# Define the neural network model
model = Sequential()

# Add input layer
model.add(Input(shape=(num_features,)))

# Add hidden layers
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=16, activation='relu'))

# Add output layer
model.add(Dense(units=1, activation='linear'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

# Print the summary of the model architecture
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,881 (11.25 KB)

 Trainable params: 2,881 (11.25 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
from sklearn.model_selection import train_test_split

# 1. Data Preparation
X = data.drop(columns=['FLOW'])  # Features
y = data['FLOW']                # Target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Model Training
#model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
#model.fit(X_train, y_train, epochs=30, batch_size=20, validation_split=0.2)
model.fit(X_train, y_train, epochs=10, batch_size=10, validation_split=0.2)

# 3. Model Evaluation
loss, mae = model.evaluate(X_test, y_test)
print("Mean Squared Error:", loss)
print("Mean Absolute Error:", mae)

# 4. Prediction
predictions = model.predict(X_test)

Epoch 1/10
4339/4339 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - loss: 0.1816 - mean_absolute_error: 0.2868 - val_loss: 0.1034 - val_mean_absolute_error: 0.2232
Epoch 2/10
4339/4339 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - loss: 0.1001 - mean_absolute_error: 0.2221 - val_loss: 0.1039 - val_mean_absolute_error: 0.2233
Epoch 3/10
4339/4339 ━━━━━━━━━━━━━━━━━━━━ 27s 6ms/step - loss: 0.0943 - mean_absolute_error: 0.2165 - val_loss: 0.0971 - val_mean_absolute_error: 0.2150
Epoch 4/10
4339/4339 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - loss: 0.0960 - mean_absolute_error: 0.2174 - val_loss: 0.0980 - val_mean_absolute_error: 0.2188
Epoch 5/10
4339/4339 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 0.0895 - mean_absolute_error: 0.2118 - val_loss: 0.0954 - val_mean_absolute_error: 0.2165
Epoch 6/10
4339/4339 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 0.0884 - mean_absolute_error: 0.2114 - val_loss: 0.0889 - val_mean_absolute_error: 0.2099
Epoch 7/10
4339/4339 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 0.0873 - mean_absolut